### Notebook to make the figure showing the classes uncovered by LCTA

In [24]:
using DataFrames
using CSV
using CairoMakie
using Statistics


In [17]:
file_path_data = "../../Data/robustness_fit_mult_0.2.CSV"

data = CSV.read(file_path_data, DataFrame)
clusters = data[:,:assignment]
k5 = data[:,:k5]
k6 = data[:,:k6]
time = data[:,:time];


In [22]:
unique_clusters = unique(clusters)

4-element Vector{Int64}:
 1
 2
 3
 4

In [ ]:
using CairoMakie, Statistics

# Extract data
clusters = data[:, :assignment]  # 500-element vector
k5 = data[:, :k5]                # 500-element vector
k6 = data[:, :k6]                # 500-element vector
time = data[:, :time]            # 500-element vector

# Reshape clusters into a 100-element vector (one cluster per person)
clusters_per_person = clusters[1:5:end]

# Get unique clusters and Wong colors
unique_clusters = unique(clusters_per_person)
colors = Makie.wong_colors()

# Set up the figure with two subplots
fig = Figure(size = (1000, 400))  # Adjusted width to accommodate the legend

# First subplot for k5
ax1 = Axis(fig[1, 1], xlabel = "τ", ylabel = "k5", 
           xticklabelsize = 14, yticklabelsize = 14, xlabelsize = 20, ylabelsize = 16, 
           xlabelfont = :bold, ylabelfont = :bold)

# Second subplot for k6
ax2 = Axis(fig[1, 2], xlabel = "τ", ylabel = "k6", 
           xticklabelsize = 14, yticklabelsize = 14, xlabelsize = 20, ylabelsize = 16, 
           xlabelfont = :bold, ylabelfont = :bold)

# Function to calculate mean and confidence intervals over time
function calculate_mean_and_ci_over_time(data, clusters, time_points)
    unique_clusters = unique(clusters)
    unique_times = unique(time_points)
    mean_values = Dict()
    ci_lower = Dict()
    ci_upper = Dict()
    
    for cluster in unique_clusters
        mean_val = Float64[]  # Explicitly typed as Vector{Float64}
        lower_val = Float64[]  # Explicitly typed as Vector{Float64}
        upper_val = Float64[]  # Explicitly typed as Vector{Float64}
        for t in unique_times
            idx = (clusters .== cluster) .& (time_points .== t)
            cluster_data = data[idx]
            if !isempty(cluster_data)
                n = length(cluster_data)
                mean_t = mean(cluster_data)
                std_t = std(cluster_data)
                ci = 1.96 * std_t / sqrt(n)  # 95% confidence interval
                push!(mean_val, mean_t)
                push!(lower_val, mean_t - ci)
                push!(upper_val, mean_t + ci)
            else
                push!(mean_val, NaN)
                push!(lower_val, NaN)
                push!(upper_val, NaN)
            end
        end
        mean_values[cluster] = mean_val
        ci_lower[cluster] = lower_val
        ci_upper[cluster] = upper_val
    end
    return mean_values, ci_lower, ci_upper, unique_times
end

# Calculate mean and CI for k5 and k6 over time
mean_k5, ci_lower_k5, ci_upper_k5, unique_times = calculate_mean_and_ci_over_time(k5, clusters, time)
mean_k6, ci_lower_k6, ci_upper_k6, _ = calculate_mean_and_ci_over_time(k6, clusters, time)

# Plot k5 with mean and confidence intervals over time
for (i, cluster) in enumerate(unique_clusters)
    lines!(ax1, unique_times, mean_k5[cluster], color = colors[i], linewidth = 2, label = "Cluster $cluster", alpha =0.8)
    band!(ax1, unique_times, ci_lower_k5[cluster], ci_upper_k5[cluster], color = (colors[i], 0.2))
end

# Plot k6 with mean and confidence intervals over time
for (i, cluster) in enumerate(unique_clusters)
    lines!(ax2, unique_times, mean_k6[cluster], color = colors[i], linewidth = 2, label = "Cluster $cluster", alpha =0.8)
    band!(ax2, unique_times, ci_lower_k6[cluster], ci_upper_k6[cluster], color = (colors[i], 0.2))
end

# Create a custom legend
legend_elements = [
    LineElement(color = colors[1], linewidth = 2),  # Cluster 1
    LineElement(color = colors[2], linewidth = 2),  # Cluster 2
    LineElement(color = colors[3], linewidth = 2),  # Cluster 3
    LineElement(color = colors[4], linewidth = 2),  # Cluster 4
]

legend_labels = ["Cluster $i" for i in 1:length(unique_clusters)]

# Add legend to the right of the plots
Legend(fig[1, 3], legend_elements, legend_labels, label_size = 16)

# Show the figure
fig

# Save the figure
save("robustness_fit_mult_0.2.png", fig)

CairoMakie.Screen{IMAGE}


In [29]:
mean_k6

Dict{Any, Any} with 4 entries:
  4 => Any[2.94633, 3.48392, 3.53891, 3.05177, 2.1766]
  2 => Any[2.98088, 3.51447, 3.52336, 3.08762, 2.14006]
  3 => Any[3.00047, 2.98259, 3.00067, 2.96315, 3.02851]
  1 => Any[3.02253, 3.02659, 3.02882, 2.99662, 2.99706]

In [26]:
mean_k5

4-element Vector{Any}:
 [0.040105604590570235]
 [0.032216787130679735]
 [0.03197222159980844]
 [0.0402026877312381]